## Семинар 13: "Обучение с подкреплением 1"

ФИО: Быстров Иван Дмитриевич

###  FrozenLake


<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>



In [1]:
pip install gym

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gym
import numpy as np 

#create a single game instance
env = gym.make("FrozenLake-v1")

#start new game
env.reset();

In [3]:
# display the game state
env.render()


SFFF
FHFH
FFFH
HFFG


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [4]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 0
printing observation:

SFFF
FHFH
FFFH
HFFG
observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [5]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 1
reward: 0.0
is game over?: False
printing new state:
  (Right)
SFFF
FHFH
FFFH
HFFG


In [6]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Попробуйте походить по замерзшему озеру, не упав в дырку. 
* Каждый шаг вы с вероятностью __0.5__ будете двигаться в выбранном направлении и с вероятностью __0.5__ в случайном.
* Если упадете, используйте __env.reset()__ чтобы перезапустить __env__

In [7]:
env.reset()

0

In [8]:
env.step(action_to_i['right'])
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


## Задание: 
Следуя шаблонам функций реализуйте алгоритм Policy iteration.
Протестируйте его с помощью функции __evaluate_policy__.
Попробуйте разные значение для gamma и сравните результаты.

In [9]:
def run_episode(env, policy, gamma = 1.0, render = False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

In [10]:
def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

In [11]:
# Используя фиксированную value_function и жадную стратегию получите policy

def extract_policy(v, gamma = 1.0):
    policy = np.zeros(16)
    for state in range(16):
        actions = np.zeros(4)
        for num_act in range(4):
            variants = env.env.P[state][num_act]
            for var in variants:
                actions[num_act] += (var[0]*(var[2] + gamma * v[var[1]]))
        policy[state] = np.argmax(actions)  
    return policy

In [12]:
# Используя фиксированную policy с помощью сэмплирования получите оценку для value_function с точностью 1e-10

def compute_policy_v(env, policy, gamma=1.0):
    v1 = np.zeros(16)
    v2 = np.zeros(16)
    eps1 = 1
    while eps1 > 1e-10:
        v1 = np.copy(v2)
        delta = -1
        for state in range(16):
            variants = env.env.P[state][policy[state]]
            v2[state] = 0
            for var in variants:
                v2[state] += (var[0]*(var[2] + gamma * v1[var[1]]))
            if abs(v1[state] - v2[state]) > delta:
                delta = abs(v1[state] - v2[state])
        eps1 = delta
    return v2

In [13]:
def policy_iteration(env, gamma = 1.0):
    policy = np.random.choice(np.arange(4), 16)
    max_iterations = 100000
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            break

        policy = new_policy
    return policy

In [14]:
gamma = [0.1*i for i in range(1,11)]
for elem in gamma:
    env = gym.make("FrozenLake-v1")
    optimal_policy = policy_iteration(env, gamma=elem)
    policy_score = evaluate_policy(env, optimal_policy, elem, n=100)
    print('policy for gamma = %.1f' %elem, ' equals ', policy_score)

policy for gamma = 0.1  equals  1.1013201111331223e-07
policy for gamma = 0.2  equals  5.286110906970636e-06
policy for gamma = 0.3  equals  9.892813848919449e-05
policy for gamma = 0.4  equals  2.49572109802786e-05
policy for gamma = 0.5  equals  0.0003040475452644388
policy for gamma = 0.6  equals  0.0008951374879878405
policy for gamma = 0.7  equals  0.003927485862694548
policy for gamma = 0.8  equals  0.013963243062539004
policy for gamma = 0.9  equals  0.07510220558012731
policy for gamma = 1.0  equals  0.74


In [15]:
gamma = 1.0
env = gym.make("FrozenLake-v1")
optimal_policy = policy_iteration(env, gamma=gamma)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=100)
print('policy for gamma = ', gamma, ' equals ', policy_score)
run_episode(env, optimal_policy, gamma, render=True)

policy for gamma =  1.0  equals  0.74

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHF

1.0